# Load dependencies

In [1]:
import numpy as np
import os
import PIL
from PIL import Image

from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam

Using TensorFlow backend.
/usr/lib64/python2.7/site-packages/cffi/model.py:532: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


# Read data

In [2]:
import os
os.chdir('')

In [11]:
size = 256
def read_image(f):
    im = Image.open(f)
    im = im.resize((size, size), PIL.Image.NEAREST)
    im = np.asarray(im, dtype='float64')
    return(im)
    
train_files = []
val_files = []
test_files = []

train_files = os.listdir('/train_images')
val_files = os.listdir('/val_images')
test_files = os.listdir('/test_images')    

In [12]:
print(train_files[:5])
print(val_files[:5])
print(test_files[:5])

['Geometric_0.jpg', 'Typography_1.jpg', 'Solid_2.jpg', 'Striped_3.jpg', 'Abstract_4.jpg']
['Typography_0.jpg', 'Striped_1.jpg', 'People and Places_2.jpg', 'Solid_3.jpg', 'Solid_4.jpg']
['Solid_0.jpg', 'Solid_1.jpg', 'Solid_2.jpg', 'Solid_3.jpg', 'Solid_4.jpg']


In [13]:
test_files[2125]

'Solid_2126.jpg'

In [ ]:
n_train_files = len(train_files)
print('Total number of train images:', n_train_files)

n_val_files = len(val_files)
print('Total number of val images:', n_val_files)

i=0
x_train = []
x_val = []
print('Reading train images ...')
for file in train_files:
    im = read_image('/train_images/'+file)
    x_train.append(im)
    i+=1

i=0
print('Reading val images ...')
for file in val_files:
    im = read_image('/val_images/'+file)
    x_val.append(im)
    i+=1

print("Done reading all images")


In [14]:
n_test_files = len(test_files)
print('Total number of test images:', n_test_files)

i=0
x_test = []
print('Reading test images ...')
for file in test_files:
    im = read_image('/test_images/'+file)
    x_test.append(im)
    i+=1

print("Done reading all images")


('Total number of test images:', 2849)
Reading test images ...
Done reading all images


In [15]:
x_train = np.array(x_train)
print(x_train.shape, 'x_train Shape')

x_val = np.array(x_val)
print(x_val.shape, 'x_val Shape')


x_test = np.array(x_test)
print(x_test.shape, 'x_test Shape')

((2849, 256, 256, 3), 'x_test Shape')


In [ ]:
np.save("train_preprocessed.npy",x_train)
np.save("val_preprocessed.npy",x_val)
#can load using np.load
# a = np.load('xyz.npy')

## Normalize the data

In [16]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.
x_val /= 255.
x_test /= 255.

In [ ]:
np.save("train_normalized.npy",x_train)
np.save("val_normalized.npy",x_val)

In [17]:
np.save("test_normalized.npy",x_test)

In [18]:
x_train = np.load('train_normalized.npy')
x_val = np.load('val_normalized.npy')
x_test = np.load('test_normalized.npy')

In [ ]:
x_test

In [19]:
y_train = []
y_val = []
y_test = []

for file in val_files:
    y_val.append(file.split('_')[0])

for file in test_files:
    y_test.append(file.split('_')[0])
        
for file in train_files:
    y_train.append(file.split('_')[0])

In [20]:
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()
y_val = le.fit_transform(y_val)
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [21]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'Checked': 3, 'Camouflage': 2, 'Sports': 15, 'Music': 10, 'Polka Dots': 12, 'Superhero': 17, 'Geometric': 7, 'People and Places': 11, 'Colourblocked': 4, 'Self Design': 13, 'Humour and Comic': 9, 'Varsity': 21, 'Floral': 6, 'Striped': 16, 'Tie and Dye': 18, 'Conversational': 5, 'Graphic': 8, 'Solid': 14, 'Tribal': 19, 'Typography': 20, 'Abstract': 0, 'Biker': 1}


In [22]:
len(set(y_val))


22

In [24]:
len(set(y_test))

22

In [25]:
y_train = to_categorical(y_train)
print(y_train.shape, 'y_train Shape')

y_val = to_categorical(y_val)
print(y_val.shape, 'y_val Shape')

y_test = to_categorical(y_test)
print(y_test.shape, 'y_test Shape')

((5649, 22), 'y_train Shape')
((997, 22), 'y_val Shape')
((2849, 22), 'y_test Shape')


## Building model using inception arch

In [26]:
# Get inception architecture from keras.applications
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras.layers import GlobalAveragePooling2D

trained_model = InceptionV3(include_top=False,weights='imagenet')
# print(trained_model.summary())

x = trained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(300,activation='relu')(x)
x = BatchNormalization()(x)
pred_inception= Dense(22,activation='softmax')(x)
model = Model(inputs=trained_model.input,outputs=pred_inception)
# print(model.summary())

In [27]:
#making the layers of inception non-trainable
for layer in trained_model.layers:
    layer.trainable=False
    
#compiling the model
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=adam)

In [16]:
model.fit(x_train, y_train, batch_size= 64, epochs= 10, validation_data=(x_val, y_val))

Train on 5649 samples, validate on 997 samples
Epoch 1/10
5649/5649 [==============================] - 26s 5ms/step - loss: 1.7314 - acc: 0.5849 - val_loss: 2.9649 - val_acc: 0.3681
Epoch 2/10
5649/5649 [==============================] - 19s 3ms/step - loss: 1.0503 - acc: 0.7123 - val_loss: 2.4173 - val_acc: 0.4243
Epoch 3/10
5649/5649 [==============================] - 19s 3ms/step - loss: 0.8293 - acc: 0.7483 - val_loss: 2.4472 - val_acc: 0.4273
Epoch 4/10
5649/5649 [==============================] - 20s 3ms/step - loss: 0.7108 - acc: 0.7816 - val_loss: 2.5750 - val_acc: 0.4514
Epoch 5/10
5649/5649 [==============================] - 19s 3ms/step - loss: 0.6107 - acc: 0.8005 - val_loss: 2.4784 - val_acc: 0.5336
Epoch 6/10
5649/5649 [==============================] - 20s 4ms/step - loss: 0.5622 - acc: 0.8143 - val_loss: 3.1639 - val_acc: 0.4293
Epoch 7/10
5649/5649 [==============================] - 20s 3ms/step - loss: 0.5285 - acc: 0.8223 - val_loss: 2.5500 - val_acc: 0.5737
Epoch 8/

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Hyper parameter tuning

In [ ]:
import itertools
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
batch_size = 64

def model_train(optimizer, lr_rate, freez, nb_epoch=2):
    tag = 'model_'+optimizer+'_lr_'+str(lr_rate)+'_freeze_'+str(freez)
    trained_model = InceptionV3(include_top=False,weights='imagenet')
    x = trained_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    pred_inception= Dense(22,activation='softmax')(x)
    model = Model(inputs=trained_model.input,outputs=pred_inception)

    #making the layers non-trainable
    for layer in trained_model.layers:
        layer.trainable=freez

    #compiling the model
    if optimizer=='adam':
        optim = Adam(lr=lr_rate)
    elif optimizer=='sgd':
        optim = SGD(lr=lr_rate)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optim)

    train_datagen = ImageDataGenerator(rescale=None, 
                                       height_shift_range=0.2, 
                                       width_shift_range=0.2,
                                       horizontal_flip=True, 
                                       rotation_range=10)
    validation_datagen = ImageDataGenerator(rescale=None)

    train_generator = train_datagen.flow(x=x_train, y=y_train,batch_size=batch_size)
    validation_generator = validation_datagen.flow(x=x_val, y=y_val,batch_size=batch_size)

    hist = model.fit_generator(train_generator,
                        steps_per_epoch=train_generator.n//batch_size, # '//' in python returns only the quotient
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.n//batch_size, 
                        verbose=0).history
    
    model.save(tag+'.h5')
    np.save(tag+'.npy', hist)
    
    t_acc = np.min(hist['acc'])
    v_acc = np.min(hist['val_acc'])
    return(t_acc, v_acc)

In [ ]:
optimizer_list = ['adam', 'sgd']
lr_rate_list = [0.01,0.001]
freez_list = [True, False]

gs_list = list(itertools.product(optimizer_list, lr_rate_list, freez_list))
print(gs_list)

In [ ]:
tr_acc = []
val_acc = []
for op, lr, fr in gs_list:
    print('Running combination - Optimizer:{}, Learning Rate:{}, Freeze Weights:{}'.format(op, lr, fr))
    r1, r2 = model_train(op, lr, fr)
    tr_acc.append(r1)
    val_acc.append(r2)

In [ ]:
import pandas as pd
df = pd.DataFrame(gs_list)
df.columns = ['Optimizer', 'Learning Rate', 'Freeze Weights']
df['Train Acc'] = tr_acc
df['Val Acc'] = val_acc

In [ ]:

r1, r2 = model_train('sgd', 0.010, True)
tr_acc.append(r1)
val_acc.append(r2)

In [42]:
from keras.models import load_model
model = load_model('model_sgd_lr_0.01_freeze_True.h5')

Exception tensorflow.python.framework.errors_impl.InvalidArgumentError: InvalidArgumentError() in <bound method _Callable.__del__ of <tensorflow.python.client.session._Callable object at 0xd77a5e10>> ignored


ResourceExhaustedError: OOM when allocating tensor of shape [] and type float
	 [[Node: batch_normalization_1361_1/moving_mean/local_step/Initializer/zeros = Const[_class=["loc:@batch_normalization_1361_1/moving_mean/Assign"], dtype=DT_FLOAT, value=Tensor<type: float shape: [] values: 0>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op u'batch_normalization_1361_1/moving_mean/local_step/Initializer/zeros', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance

# tried genetic algo

In [32]:
    import random
    import gentun

    from sklearn.preprocessing import LabelBinarizer
    from gentun import Population, GeneticCnnIndividual, RussianRouletteGA

    n = x_train.shape[0]
    selection = random.sample(range(n), 10000)  # Use only a subsample
    x_train = x_train.reshape(n, 28, 28, 1)[selection]

    pop = Population(
        GeneticCnnIndividual, x_train, y_train, size=20, crossover_rate=0.3, mutation_rate=0.1,
        additional_parameters={
            'kfold': 5, 'epochs': (20, 4, 1), 'learning_rate': (1e-3, 1e-4, 1e-5), 'batch_size': 32
        }, maximize=True
    )
    ga = RussianRouletteGA(pop, crossover_probability=0.2, mutation_probability=0.8)
    ga.run(50)

ImportError: No module named gentun

## Building a custom model

In [37]:
from keras.models import Sequential
model = Sequential()
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', 
                        input_shape=(256,256,3), 
                        activation='relu'))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(22, activation='softmax'))

In [38]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [41]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_val, y_val),
          shuffle=True)

Train on 5649 samples, validate on 997 samples
Epoch 1/10


Exception tensorflow.python.framework.errors_impl.InvalidArgumentError: InvalidArgumentError() in <bound method _Callable.__del__ of <tensorflow.python.client.session._Callable object at 0xd77b0650>> ignored


ResourceExhaustedError: OOM when allocating tensor of shape [] and type float
	 [[Node: loss_1/mul/x = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [] values: 1>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))